In [1]:
import sys

if sys.prefix != sys.base_prefix:
    print(f"Python is running inside a virtual environment {sys.prefix}.")
else:
    print(f"Python is running outside a virtual environment {sys.base_prefix}.")


Python is running inside a virtual environment c:\UMD\Sem 1\MSML602 Principles in Data Science\.venv.


In [1]:
import cupy as cp

In [6]:
!pip install cuml

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [47 lines of output]
      running bdist_wheel
      running build
      c:\UMD\Sem 1\MSML602 Principles in Data Science\.venv\Lib\site-packages\setuptools\_distutils\cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
      !!
      
              ********************************************************************************
              Please avoid running ``setup.py`` directly.
              Instead, use pypa/build, pypa/installer or other
              standards-based tools.
      
              See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
              ********************************************************************************
      
      !!
        self.initialize_options()
      installing to build\bdist.win-amd64\wheel
      running install
      Traceback (most recent call last):
        F


  Using cached cuml-0.6.1.post1.tar.gz (1.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for cuml
Failed to build cuml


In [2]:
from cuml import TruncatedSVD

ModuleNotFoundError: No module named 'cuml'

In [2]:
import pandas as pd
import numpy as np
import os
import gzip
import copy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.decomposition import TruncatedSVD

In [3]:
base_path = 'C:\\UMD\\Sem 1\\MSML602 Principles in Data Science\\Assignment 4'
gzip_file_name = f'{base_path}\\finefoods.txt.gz'
file_name = f'{base_path}\\finefoods.txt'
if not os.path.exists(gzip_file_name):
    os.system("wget https://snap.stanford.edu/data/finefoods.txt.gz")

binary_data = gzip.open(gzip_file_name, 'r').read()

In [4]:
with gzip.open(gzip_file_name, 'r') as file: lines = file.readlines()

# Initialize an empty list to store dictionaries
data = []

# Initialize a dictionary to store key-value pairs for each paragraph
current_paragraph = {}
last_key = ''

# Loop through each line
for line in lines:
    # If the line is not empty, split it into key and value
    line = line.decode('Windows-1252').strip()
    if line:
        line_data = line.split(":", 1)
        assert len(line_data) <= 2, f"More than 2 elements, {line_data}"
        if len(line_data) == 1:
            current_paragraph[last_key] += line_data[0]
        else:
            key, value = line_data
            current_paragraph[key] = value.strip()
            last_key = key
    else:
        # If the line is empty, it indicates the end of a paragraph
        # Append the dictionary to the data list and reset for the next paragraph
        data.append(current_paragraph)
        current_paragraph = {}

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)

# Display the resulting DataFrame
print(df)


       product/productId   review/userId               review/profileName  \
0             B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1             B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2             B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3             B000UA0QIQ  A395BORC6FGVXV                             Karl   
4             B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   
...                  ...             ...                              ...   
568449        B001EO7N10  A28KG5XORO54AY                 Lettie D. Carter   
568450        B003S1WTCU  A3I8AFVPEE8KI5                        R. Sawyer   
568451        B004I613EE  A121AA1GQV751Z                    pksd "pk_007"   
568452        B004I613EE   A3IBEVCTXKNOH          Kathy A. Welch "katwel"   
568453        B001LR2CU2  A3LGQPJCZVL9UC                         srfell17   

       review/helpfulness review/score review/time  \
0                    

In [5]:
df['review/score'] = df['review/score'].astype(float).astype(int)
df['review/time'] = pd.to_datetime(df['review/time'].astype(int), unit='s')

In [6]:
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(df['review/text'])

When I split `X_tfidf` into training and test data with an 80%, 20% split (assume `X_train` and `X_test` respectively), the training data is of 407. GiB and an array with shape (454763, 120252) and each data type is float64. Generally, gaussian requires matrix and passing this enormous amount of data is not possible. 

So, one needs to do PCA or SVD and then take the most important dataset to do GaussianNB.

In [9]:
X_tfidf.shape

(568454, 120252)

In [16]:
svd = TruncatedSVD(n_components=1000)
X_svd = svd.fit_transform(X_tfidf)
explained_variance = np.sum(svd.explained_variance_ratio_)
print(X_svd.shape, explained_variance)

(568454, 1000) 0.5721161653797121


In [19]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_svd, df['review/score'], test_size=0.2, random_state=42)

# Train Gaussian Naive Bayes
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [21]:
# Make predictions on the test set
predictions = classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy:.2f}')

# Display classification report
print('Classification Report:')
print(classification_report(y_test, predictions))

Accuracy: 0.48
Classification Report:
              precision    recall  f1-score   support

           1       0.19      0.57      0.28     10326
           2       0.15      0.14      0.14      5855
           3       0.19      0.22      0.21      8485
           4       0.26      0.21      0.23     16123
           5       0.77      0.58      0.66     72902

    accuracy                           0.48    113691
   macro avg       0.31      0.34      0.31    113691
weighted avg       0.57      0.48      0.51    113691



In [7]:
svd2 = TruncatedSVD(n_components=2000)
X_svd2 = svd2.fit_transform(X_tfidf)
explained_variance2 = np.sum(svd2.explained_variance_ratio_)
print(X_svd2.shape, explained_variance2)

: 

In [ ]:
# Split the data into training and testing sets
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_svd2, df['review/score'], test_size=0.2, random_state=42)

# Train Gaussian Naive Bayes
classifier2 = GaussianNB()
classifier2.fit(X_train2, y_train)

In [ ]:
# Make predictions on the test set
predictions2 = classifier2.predict(X_test2)

# Evaluate the classifier
accuracy2 = accuracy_score(y_test2, predictions2)
print(f'Accuracy: {accuracy2:.2f}')

# Display classification report
print('Classification Report:')
print(classification_report(y_test2, predictions2))